## N-Gram

For this solution we use the N-Gram model approach. The steps will be as follows:

1. Ingest data
2. Generate vocab, encoder, decoder
3. Generate a count dictionary of all n-grams generated from our data. This will serve as our frequency lookup table.
4. Read test data.
5. Iteratively for each character:

  * generate a bigram matching the length specified earlier. Where necessary, using padding character to maintain length.

  * Use our frequency lookup table to return count of the current bigram 

  * Divide by total no of occurences of all bigrams 

  * Normalize these probabilities and return the probability matrix generated

  * Look up generated probability of our expected character

  * calculate log of the value and subtract from overall loss of the entire model

  * remove first character of our bigram and append current character to the end of the bigram


Part of the code modified from: https://www.youtube.com/watch?v=zz1CFBS4NaY



In [1]:
def suggest_next_char(input_, ngram_counts, vocab_counts, vocab, encoder, len_gram, laplace_weight=0):

    # Consider the last bigram of sentence
    tokenized_input = [x.lower() for x in input_]
    last_gram = tokenized_input[-(len_gram):]

    # print("last gram", last_gram)
    
    # # Calculating probability for each char in vocab
    vocab_probabilities = {}

    total_ngrams = sum(ngram_counts.values())
    total_vocab = sum(vocab_counts.values())

    for vocab_char in vocab:

        original_gram = test_gram = [last_gram[x] for x in range(len_gram)]
        test_gram.pop()
        test_gram.insert(0,str(vocab_char))
        likelihood_gram = tuple(test_gram)
        likelihood_gram_count = ngram_counts.get(likelihood_gram, 0)


        test_gram.pop(0)
        test_gram.append(str(vocab_char))
        test_gram = tuple(test_gram)
        test_gram_count = ngram_counts.get(test_gram, 0)

        # Naive Bayes probability
        # posterior = (likelihood * prior)/evidence
        likelihood = likelihood_gram_count / total_ngrams
        prior = vocab_counts[vocab_char] / total_vocab
        evidence = ngram_counts.get(tuple(original_gram), 0) / total_ngrams

        probability = ((likelihood * prior) + laplace_weight)/ (evidence + laplace_weight)

        vocab_probabilities[vocab_char] = probability

    # top_suggestions = sorted(vocab_probabilities.items(), key=lambda x: x[1], reverse=True)
    top_suggestions = list(vocab_probabilities.items())

    totals = sum(map(lambda x: x[1], top_suggestions))
    coeff = 1/totals if totals != 0 else 1

    top_suggestions_standardized =  [tuple([x[0], x[1] * coeff]) for x in top_suggestions]

    return top_suggestions_standardized

In [2]:
from math import log2, inf

def evaluate_one(input, ngram_counts, vocab_counts, vocab, encoder,len_gram, laplace_weight=0):

  input = input.lower()

  max_history = 10 #max sentence length
  history = ['<S>'] * len_gram
  loss_anything_goes = 0
  count = 0

  for ind, c in enumerate(input):

    count += 1

    loss_anything_goes -= log2(predict_next_proba(c, history, ngram_counts, vocab_counts, vocab, encoder, len_gram, laplace_weight))

    if len(history) == max_history:
      history.pop(0)

    history.append(c)
    
  return [loss_anything_goes/count]


def predict_next_proba(c, history, ngram_counts, vocab_counts, vocab, encoder, len_gram, laplace_weight=0):

  # pass our x into the model and return a prediction matrix
  y_pred = suggest_next_char(history, ngram_counts, vocab_counts, vocab, encoder, len_gram, laplace_weight)

  # print("Expected ", c)
  # print("top_suggestions", y_pred, " expected char prob: ", y_pred)
  # print("\n Char: ", c, " Predictions: ", y_pred, " Char prob", y_pred[encoder[c]][1])

  # return the computed probability of our character
  proba = y_pred[encoder[c]][1]

  return proba

## Test Swahili

### Ingest Data

In [11]:
import numpy as np

path_train = r"/content/sw-train.txt"
path_test = r"/content/sw-test.txt"

corpus = open(path_train).readlines()[0]

corpus = corpus.replace("\n", " ")

lower_case_corpus = [w.lower() for w in corpus]

len_gram = 4

for x in range(len_gram):
  lower_case_corpus.insert(0, '<S>')


vocab = sorted(list(set(lower_case_corpus)))
# vocab.append('<S>')
encoder = dict((c,i) for i,c in enumerate(vocab))
decoder = dict((i,c) for i,c in enumerate(vocab))

In [20]:
ngram_counts = {}
vocab_counts = {}
vocab_counts['<S>'] = len_gram - 1

# Sliding through corpus to get ngram counts
for i in range(len(lower_case_corpus) - (len_gram-1)):

    # Getting ngram
    end = i + len_gram
    new_char = lower_case_corpus[end-1]

    # Track vocab counts
    if new_char in vocab_counts.keys():
      vocab_counts[new_char] += 1
    else:
      vocab_counts[new_char] = 1

    ngram = tuple([lower_case_corpus[x] for x in range(i, end)])

    # Keeping track of the ngram counts
    if ngram in ngram_counts.keys():
        ngram_counts[ngram] += 1
    else:
        ngram_counts[ngram] = 1
    

### Evaluate against test data

In [23]:
input = open(path_test).readlines()
print("Swahili cross entropy loss: ", evaluate_one(input[0][0:50000], ngram_counts, vocab_counts, vocab, encoder, len_gram, laplace_weight=1))

Swahili cross entropy loss:  [5.614758135501889]


## Test Kwere

### Ingest Data

In [5]:
import numpy as np

path_train_ = r"/content/cwe-train.txt"
path_test_ = r"/content/cwe-test.txt"

corpus_ = open(path_train_).readlines()[0]

corpus_ = corpus_.replace("\n", " ")

lower_case_corpus_ = [w.lower() for w in corpus_]

len_gram_ = 4

for x in range(len_gram_):
  lower_case_corpus_.insert(0, '<S>')


vocab_ = sorted(list(set(lower_case_corpus_)))
encoder_ = dict((c,i) for i,c in enumerate(vocab_))
decoder_ = dict((i,c) for i,c in enumerate(vocab_))

In [6]:
ngram_counts_ = {}
vocab_counts_ = {}
vocab_counts_['<S>'] = len_gram_-1

# Sliding through corpus to get ngram counts
for i in range(len(lower_case_corpus_) - (len_gram_-1)):

    # Getting ngram
    end = i + len_gram_
    new_char = lower_case_corpus_[end-1]

    # Track vocab counts
    if new_char in vocab_counts_.keys():
      vocab_counts_[new_char] += 1
    else:
      vocab_counts_[new_char] = 1
      

    ngram = tuple([lower_case_corpus_[x] for x in range(i, end)])
    # print(end,ngram)
    # Keeping track of the ngram counts
    if ngram in ngram_counts_.keys():
        ngram_counts_[ngram] += 1
    else:
        ngram_counts_[ngram] = 1
    

### Evaluate against test data

In [7]:
input_ = open(path_test_).readlines()
input_[0] = input_[0].replace("\n", " ")
print("Kwere cross entropy loss: ", evaluate_one(input_[0], ngram_counts_, vocab_counts_, vocab_, encoder_,len_gram = len_gram_,laplace_weight = 1))

Kwere cross entropy loss:  [4.999983825016389]


## Generate sentences

In [ ]:
input_sent = 'maji katika'
sent_len = 100

for i in range(sent_len):
  next_char = suggest_next_char(input_sent, ngram_counts_, vocab, encoder)
  print(input_sent)
  input_sent = input_sent + next_char[0][0]

print(input_sent)



In [ ]:
x = [0, 1, 2]

In [ ]:
y = x.insert(0, 3)

In [ ]:
x.pop(0)

3

In [ ]:
x

[0, 1, 2]